In [6]:
import json
from tqdm import tqdm

with open("structured_bios_new_english_fixed.json", "r", encoding="utf-8") as f:
    people = json.load(f)

In [7]:
def years_overlap(y1, y2):
    """iki yıl aralığı çakışıyor mu (örn: 2005-2010 vs 2008-2012)"""
    try:
        s1, e1 = [int(y) for y in y1.split("-")]
        s2, e2 = [int(y) for y in y2.split("-")]
        return max(s1, s2) <= min(e1, e2)
    except:
        return False


In [8]:
results = []

for i in tqdm(range(len(people))):
    for j in range(i + 1, len(people)):
        p1 = people[i]
        p2 = people[j]
        rels = []

        # AYNI_MEMLEKETTEN
        if p1["dogum_yeri"] == p2["dogum_yeri"]:
            rels.append("AYNI_MEMLEKETTEN")

        # OKUL ilişkileri (ve yıl çakışması)
        for okul, yil in [("ilkokul", "ilkokul_yillari"), ("lise", "lise_yillari"),
                          ("universite", "universite_yillari"), ("yuksek_lisans", "yuksek_lisans_yillari"),
                          ("doktora", "doktora_yillari")]:
            if p1.get(okul) and p1.get(okul) == p2.get(okul):
                if p1.get(yil) and p2.get(yil) and years_overlap(p1[yil], p2[yil]):
                    rel_ad = okul.upper() + "_ARKADASI"
                    rels.append(rel_ad)

        # MESLEKTAŞ
        if p1.get("bolum") and p1.get("bolum") == p2.get("bolum"):
            rels.append("MESLEKTAS")

        # AYNI_KURUMDA_CALISMIŞ
        ortak_kurum = set(p1.get("calistigi_kurumlar", [])) & set(p2.get("calistigi_kurumlar", []))
        if ortak_kurum:
            rels.append("AYNI_KURUMDA_CALISMIS")

        # İLİŞKİ VARSA HER İKİ YÖNDE DE EKLE
        if rels:
            # p1 -> p2 yönünde
            results.append({
                "source": p1["ad"],
                "target": p2["ad"],
                "relations": rels
            })
            
            # p2 -> p1 yönünde (ters yön)
            results.append({
                "source": p2["ad"],
                "target": p1["ad"],
                "relations": rels
            })
            


100%|██████████| 243/243 [00:00<00:00, 386.19it/s]


In [9]:
"""# Sonuçları kaydet
with open("automated_relationships.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"{len(results)} ilişki bulundu.")"""

'# Sonuçları kaydet\nwith open("automated_relationships.json", "w", encoding="utf-8") as f:\n    json.dump(results, f, ensure_ascii=False, indent=2)\n\nprint(f"{len(results)} ilişki bulundu.")'

In [10]:
# Sonuçları kaydet - Her relation türü için ayrı CSV dosyası
import csv
from collections import defaultdict

# Her relation türü için ayrı liste oluştur
relation_data = defaultdict(list)

for result in results:
    for relation in result["relations"]:
        relation_data[relation].append({
            "source": result["source"],
            "target": result["target"]
        })

# Her relation türü için ayrı CSV dosyası oluştur
for relation_type, data in relation_data.items():
    filename = f"relation_{relation_type.lower()}.csv"
    
    with open(filename, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        
        # Başlık satırı
        writer.writerow(["source", "target", "relation_type"])
        
        # Veri satırları (zaten iki yönlü olarak eklenmiş)
        for item in data:
            writer.writerow([item["source"], item["target"], relation_type])
    
    print(f"✅ {filename} oluşturuldu - {len(data)} ilişki")

# Ana CSV dosyası da oluştur (tüm ilişkiler)
with open("automated_relationships_all.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    
    # Başlık satırı
    writer.writerow(["source", "target", "relation_type"])
    
    # Tüm ilişkileri tek tek yaz
    for result in results:
        for relation in result["relations"]:
            writer.writerow([result["source"], result["target"], relation])

print(f"\n📊 Toplam {len(results)} ilişki bulundu.")
print("📁 Oluşturulan dosyalar:")
print("- automated_relationships_all.csv (tüm ilişkiler)")

# Her relation türü için dosya listesi
unique_relations = set()
for result in results:
    unique_relations.update(result["relations"])

for relation in sorted(unique_relations):
    print(f"- relation_{relation.lower()}.csv")

✅ relation_lise_arkadasi.csv oluşturuldu - 3188 ilişki
✅ relation_ayni_kurumda_calismis.csv oluşturuldu - 6970 ilişki
✅ relation_universite_arkadasi.csv oluşturuldu - 6426 ilişki
✅ relation_ayni_memleketten.csv oluşturuldu - 6508 ilişki
✅ relation_meslektas.csv oluşturuldu - 3742 ilişki
✅ relation_yuksek_lisans_arkadasi.csv oluşturuldu - 3028 ilişki
✅ relation_ilkokul_arkadasi.csv oluşturuldu - 210 ilişki
✅ relation_doktora_arkadasi.csv oluşturuldu - 22 ilişki

📊 Toplam 21512 ilişki bulundu.
📁 Oluşturulan dosyalar:
- automated_relationships_all.csv (tüm ilişkiler)
- relation_ayni_kurumda_calismis.csv
- relation_ayni_memleketten.csv
- relation_doktora_arkadasi.csv
- relation_ilkokul_arkadasi.csv
- relation_lise_arkadasi.csv
- relation_meslektas.csv
- relation_universite_arkadasi.csv
- relation_yuksek_lisans_arkadasi.csv
